In [1]:
import stanza, re, benepar, psutil, gc, json, csv, time, string, os

import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns

from textstat import textstat
from datetime import datetime
from collections import Counter

import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk import Tree
from textblob import Word
from spellchecker import SpellChecker

import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator, LogFormatter

In [2]:
source_texts= list()
source_texts.append("Kazuo Ishiguro - Never Let Me Go")
source_texts.append("Kazuo Ishiguro - The Remains of the Day")
source_texts.append("Kazuo Ishiguro - A Pale View of Hills-Knopf Doubleday Publishing Group (1990)")
source_texts.append("Kazuo-Ishiguro-When-We-Were-Orphans-Alfred-A.-Knopf_Vintage-_2001_")
source_texts.append("The Buried Giant (Kazuo Ishiguro) (Z-Library)-1")
source_texts.append("Kazuo Ishiguro - The Unconsoled-Vintage (1996)")
directory_path = "C:/Users/Roland/Documents/AI/stylometry/"


print(file_path_edited_2)

print(file_path_edited_2)

Select choice of text by specifying index (starting with 0):

In [91]:
text_choice = 5

file_path = directory_path+source_texts[text_choice]+" edited_2.txt"
if os.path.isfile(file_path):
    print ("file found at ", file_path)
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
else:
    print("file not found at ", file_path)
    file_path = directory_path+source_texts[text_choice]+" edited_1.txt"
    if os.path.isfile(file_path):
        print ("file found at ", file_path)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
    else:
        print("file not found at ", file_path)
        file_path = directory_path+source_texts[text_choice]+" edited.txt"
        if os.path.isfile(file_path):
            print ("file found at ", file_path)
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
        else:
            print("file not found at ", file_path)


file found at  C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Unconsoled-Vintage (1996) edited_2.txt


In [92]:
valid_char_pattern = re.compile(r'[a-zA-Z0-9\s.:;,?!\'"()£\-—…&]')

re_chars = list()
invalid_chars = set()
for i, char in enumerate(text):
    if not valid_char_pattern.match(char):
        invalid_chars.add(char)

# Output the list of invalid characters and their positions
invalid_char_positions = [(char, [i for i, c in enumerate(text) if c == char]) for char in invalid_chars]
print(f"invalid characters using re: {invalid_chars}")
#print(invalid_char_positions)
re_chars = invalid_chars

invalid characters using re: {'é'}


In [93]:
valid_chars = string.ascii_letters + string.digits + string.punctuation + "…—àêé \n\t"
valid_chars = ''.join([char for char in valid_chars if char not in "{}~$%"])
print (f"valid characters used by string: {string.ascii_letters + string.digits + ' ' + string.punctuation}")

string_chars = list()
invalid_chars = set()
for i, char in enumerate(text):
    if char not in valid_chars:
        invalid_chars.add(char)
    #invalid_chars.difference_update({"\n","\t"})

invalid_char_positions = [(char, [i for i, c in enumerate(test_text) if c == char]) for char in invalid_chars]
print(f"invalid characters found using string: {invalid_chars}")

valid characters used by string: abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
invalid characters found using string: set()


In [81]:
def find_capitalized_words(text):
    """
    Finds all words in the string that are in capitals, excluding words of one character.
    
    Parameters:
    - text: The input string.
    
    Returns:
    - A list of fully capitalized words with more than one character.
    """
    # Use regex to match words with all capital letters and exclude single-character words
    capitalized_words = re.findall(r'\b[A-Z]{2,}\b', text)
    return capitalized_words

In [94]:
capitalized_words = find_capitalized_words(text)
for word in capitalized_words:
    if word != "OK":
        ind = text.find(word)
        window = text[ind-75:ind+40]
        window = window.replace("\n", " ")
        window = window.replace("\t", " ")
        print(window)


uld enjoy pretending the teams were 'real' ones, such as Ajax Amsterdam or AC Milan, and had given the teams his ow
he film in which the astronauts first suspect the motives of the computer, HAL, central to every aspect of life abo
he film in which the astronauts first suspect the motives of the computer, HAL, central to every aspect of life abo
he film in which the astronauts first suspect the motives of the computer, HAL, central to every aspect of life abo
 street, resolving to call on her in the morning at the first opportunity. II     21     I awoke to find bright sun
 street, resolving to call on her in the morning at the first opportunity. II     21     I awoke to find bright sun
 street, resolving to call on her in the morning at the first opportunity. II     21     I awoke to find bright sun
e entries grew facetious. Someone was claiming to have lost 'three million US dollars'. Another entry was that of '
t see them. They had not been seated in any of the boxes nor in any of t

In [95]:
hyphenated_words = re.findall(r"(\w+)-\n(\w+)", text)

# Print the hyphenated words
for first_part, second_part in hyphenated_words:
    print(f"Found broken word: {first_part}-{second_part}")
print(len(hyphenated_words))


Found broken word: not-so
Found broken word: so-called
Found broken word: mock-pompous
Found broken word: near-darkness
Found broken word: pudgy-faced
Found broken word: long-range
Found broken word: sun-filled
Found broken word: full-chested
Found broken word: nervous-looking
Found broken word: red-nosed
Found broken word: have-no
Found broken word: half-way
Found broken word: high-pitched
Found broken word: mid-sentence
Found broken word: self-possessed
Found broken word: half-finished
Found broken word: early-morning
17


In [96]:
display_invalid_chars(text)

Invalid character 'é' (233) at position 10485:
t, that's true, but we've often talked it over - we meet every Sunday afternoon at the Hungarian Caf[é] in the Old Town, you could come and join us, you'd be a most welcome guest, sir - well, we've ofte

Invalid character 'é' (233) at position 10843:
 ones who came after us, of course, they take it all for granted. But our group at the Hungarian Caf[é], we know we've made a difference, even if it's a small one. You'd be very welcome to join us, sir.

Invalid character 'é' (233) at position 11455:
 it. But to return to what I was saying, we've been discussing this topic a lot at the Hungarian Caf[é]. I mean about these old resolutions we each made all those years ago. You see, none of us thought 

Invalid character 'é' (233) at position 12413:
eriod of ill-health, and I was finding things difficult, and so we discussed it at the Hungarian Caf[é]. Well, in the end, my colleagues all agreed there was no need for me to be so strict on myself. 

In [54]:
# further_corrections = [dict(), dict(), dict(), dict(), dict(), dict()]
# Don't run if already contains useful data

In [70]:
further_corrections[text_choice]["a busy time/"] = "a busy time"
further_corrections[text_choice]["[said, getting to my feet"] = "I said, getting to my feet"
further_corrections[text_choice]["[go"] = "I go"
further_corrections[text_choice]["[fit's a boy"] = "If it's a boy"
further_corrections[text_choice]["[care even less"] = "I care even less"
further_corrections[text_choice]["[asked"] = "I asked"
further_corrections[text_choice]["[may just catch"] = "I may just catch"
further_corrections[text_choice]["[here for years"] = "here for years"
further_corrections[text_choice]["[opened my door"] = "I opened my door"
further_corrections[text_choice]["[ I would like to have"] = " I would like to have"
further_corrections[text_choice]['] said. "1 ironed'] = 'I said. "I ironed'
further_corrections[text_choice]["of_ those killed"] = "of those killed"
further_corrections[text_choice]["I dà want you to"] = "I'd want you to"
further_corrections[text_choice]["Ogata-Sàn"] = "Ogata-San"
further_corrections[text_choice]["lifted itàff"] = "lifted it off"

In [71]:
text_edited = replace_strings_with_context(text, further_corrections[text_choice], window = 20, silent = False)

Before: u caught me at such [a busy time/]' my husband said. "
After:  u caught me at such [a busy time]' my husband said. "

Before: should have asked," [[said, getting to my feet]. "You won't always 
After:  should have asked," [I said, getting to my feet]. "You won't always 

Before:  really don't mind. [[fit's a boy] we could name him a
After:   really don't mind. [If it's a boy] we could name him a

Before: e that thought, and [[care even less] now." We stopped wa
After:  e that thought, and [I care even less] now." We stopped wa

Before: day today, Mariko?" [[asked]. "Did you do any dr
After:  day today, Mariko?" [I asked]. "Did you do any dr

Before: new house, Father?" [[asked]. . "My house? Why, 
After:  new house, Father?" [I asked]. . "My house? Why, 

Before: home, in which case [[may just catch] him. That would be 
After:  home, in which case [I may just catch] him. That would be 

Before: ou lived and worked [[here for years], Ogata-San. Don't w
After:  ou lived and worke

In [72]:
display_invalid_chars(text_edited)

In [97]:
print(text[:5000])






The taxi driver seemed embarrassed to find there was no one - not even a clerk behind the reception desk
- waiting to welcome me. He wandered across the deserted lobby, perhaps hoping to discover a staff
member concealed behind one of the plants or armchairs. Eventually he put my suitcases down beside the
elevator doors and, mumbling some excuse, took his leave of me.

 The lobby was reasonably spacious, allowing several coffee tables to be spread around it with no
sense of crowding. But the ceiling was low and had a definite sag, creating a slightly claustrophobic
mood, and despite the sunshine outside the light was gloomy. Only near the reception desk was there a
bright streak of sun on the wall, illuminating an area of dark wood panelling and a rack of magazines in
German, French and English. I could see also a small silver bell on the reception desk and was about to go
over to shake it when a door opened somewhere behind me and a young man in uniform appeared.

 'Good afternoo

In [98]:
print(text[-5000:])

about
all your troubles. Or if you prefer, we can just forget about it all and talk about whatever you like,
whatever's likely to cheer you up. Football, cinema. Anything you like. But the first thing to do is to get
some breakfast. You look like you haven't eaten for some time.'

 'You're quite right,' I said. 'Now I think of it, I haven't eaten for a very long time. But please tell
me. Where is this tram going? I have to get to my hotel to pack my things. You see, I have a flight this
morning to Helsinki. I have to get to my hotel pretty soon.'

 'Oh, this tram will get you more or less anywhere you like in the city. This is what we call the
morning circuit. Then there's the evening circuit. Twice a day a tram goes right the way round the entire
circuit. Oh yes, you can go anywhere on this tram. It's the same again in the evening, but the atmosphere's
quite different then. Oh yes, this is a marvellous tram.'

 'How splendid. Well then, excuse me. I think I'll take up your suggestion 

In [103]:
display_text_in_context(text, "20", window = 90, repeat = True)

191156 t least half an hour, and I saw with some relief it
was to be the science fiction classic 2001: A Space Odyssey - a favourite of mine which I never tired of
seeing.

 As soon as thos
593812 led into a corridor, I
stepped quickly through and gestured urgently to Sophie and Boris.
20




I closed the door behind them and we all three looked about us. With some triumph I s


In [76]:
print(file_path) 

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - A Pale View of Hills-Knopf Doubleday Publishing Group (1990) edited_1.txt


In [77]:
with open(file_path, 'w', encoding='utf-8') as f:
    f.write(text_edited)

In [109]:
print(ord(text[593814]))

10


In [189]:
display_text_in_context(text, "I\n\n\n\n\n11")
display_text_in_context(text, "II\n\n\n\n\n21")
display_text_in_context(text, "V\n\n\n\n\n28")
for i in range(1, 40):
    display_text_in_context(text, str(i)+chr(10), repeat = False)




lk slowly away in the direction of the lobby.     I     11     I was woken by the ringing of the telephone o
l on her in the morning at the first opportunity. II     21     I awoke to find bright sunlight pouring in th
et the proprietor's wife had left out for me.     V     28     I awoke with the panicky sense that I had sle
ly away in the direction of the lobby.     I     11     I was woken by the ringing of the telephone on
t myself sliding into a deep and exhausted sleep. 2     When I was roused by the bedside telephone, I 
ow how I get on.'   'Thank you so much, sir.'     3     The route from the hotel to the Old Town - a w
f the swing park into a deserted back street.     4 It was clear we had now left behind the Old Town. 
n began to make our way over to the bus stop.     5     We stood beneath the street lamp for several m
man comes back, we'll tell him our new plan.'     6     Just at this point a movement caught my eye an
Stephan before leading him towards the hotel.     7 Th

In [201]:
further_corrections[text_choice] = dict()
further_corrections[text_choice]["I\n\n\n\n\n11"] = ""
further_corrections[text_choice]["II\n\n\n\n\n21"] = ""
further_corrections[text_choice]["V\n\n\n\n\n28"] = ""
for i in range(10,39):
    if i not in [11, 21, 28]:
        further_corrections[text_choice][str(i)+"\n"] = ""
for i in range(10):
    further_corrections[text_choice][str(i)+"\n"] = ""


In [176]:
for i in range(16):
    print (320678+i, ord(text[320678+i]), text[320678+i])

320678 10 

320679 73 I
320680 10 

320681 10 

320682 10 

320683 10 

320684 10 

320685 49 1
320686 49 1
320687 10 

320688 10 

320689 10 

320690 10 

320691 10 

320692 73 I
320693 32  


In [202]:
text_edited = replace_strings_with_context(text, further_corrections[text_choice], window = 20, silent = False)

Before: n of the lobby.     [I




11]     I was woken by 
After:  n of the lobby.     []     I was woken by 

Before:  first opportunity. [II




21]     I awoke to find
After:   first opportunity. []     I awoke to find

Before: eft out for me.     [V




28]     I awoke with th
After:  eft out for me.     []     I awoke with th

Before: o a deep sleep.     [10
]    I had not been a
After:  o a deep sleep.     []    I had not been a

Before: isplay cabinet.     [12
]    When I came back
After:  isplay cabinet.     []    When I came back

Before: n the carriage.     [13
]    The tram rattled
After:  n the carriage.     []    The tram rattled

Before:  my eyes again.     [14
]    We had slowed ri
After:   my eyes again.     []    We had slowed ri

Before: o the sunshine.     [15
]    We boarded the b
After:  o the sunshine.     []    We boarded the b

Before: led pleasantly.     [16
]    The woman who ha
After:  led pleasantly.     []    The woman who ha

Before: omposed manner.     [1

In [203]:
print(len(text_edited))
print(len(text))

1134964
1135083


In [204]:
print(file_path)

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Unconsoled-Vintage (1996) edited_2.txt


In [205]:
with open(file_path, 'w', encoding='utf-8') as f:
    f.write(text_edited)

In [162]:
len("110110100100000001010010010101110010101111001")

45

In [4]:
file_path = directory_path+source_texts[text_choice]+".txt"
file_path_edited = file_path[:-4]+" edited.txt"
file_path_edited_1 = file_path[:-4]+" edited_1.txt"
file_path_edited_2 = file_path[:-4]+" edited_2.txt"
print (file_path)
print(file_path_edited)
print(file_path_edited_1)
print(file_path_edited_2)

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Unconsoled-Vintage (1996).txt
C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Unconsoled-Vintage (1996) edited.txt
C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Unconsoled-Vintage (1996) edited_1.txt
C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Unconsoled-Vintage (1996) edited_2.txt


Try reading the file with different encodings:

In [6]:
encodings = ['utf-8', 'ascii', 'latin-1','windows-1252', 'iso-8859-1', 'iso-8859-15']
texts = []
valid_encodings = list()
for i, encoding in enumerate(encodings):
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            test_text = file.read()
        print("""Successfully read the file with encoding: {encoding} (encoding index {ind}).  This gives the beginning as:\n{content}\n""".format(encoding=encoding, ind=i, content=test_text[0:200].replace("\n", "  ")))
        texts.append(test_text)
        valid_encodings.append(i)
    except UnicodeDecodeError:
        print(f"Failed to read with encoding: {encoding}")
print("valid encodings: ",valid_encodings)

Successfully read the file with encoding: utf-8 (encoding index 0).  This gives the beginning as:
    PART I      The taxi driver seemed embarrassed to find there was no one - not even a clerk behind the reception desk  - waiting to welcome me. He wandered across the deserted lobby, perhaps hoping to di

Failed to read with encoding: ascii
Successfully read the file with encoding: latin-1 (encoding index 2).  This gives the beginning as:
    PART I      The taxi driver seemed embarrassed to find there was no one - not even a clerk behind the reception desk  - waiting to welcome me. He wandered across the deserted lobby, perhaps hoping to di

Successfully read the file with encoding: windows-1252 (encoding index 3).  This gives the beginning as:
    PART I      The taxi driver seemed embarrassed to find there was no one - not even a clerk behind the reception desk  - waiting to welcome me. He wandered across the deserted lobby, perhaps hoping to di

Successfully read the file with encod

Are they the same?

In [7]:
for i in range(len(texts)):
    print (f"text {i} length {len(texts[i])}:-")
    for j, t in enumerate(texts):
        print (j, t == texts[i])

text 0 length 1159560:-
0 True
1 False
2 False
3 False
4 False
text 1 length 1159650:-
0 False
1 True
2 False
3 True
4 False
text 2 length 1159650:-
0 False
1 False
2 True
3 False
4 False
text 3 length 1159650:-
0 False
1 True
2 False
3 True
4 False
text 4 length 1159650:-
0 False
1 False
2 False
3 False
4 True


Now load text file with selected encoding

In [9]:
def check_for_file(file_path):
    if os.path.isfile(file_path):
        print("The file exists.")
    else:
        print("The file does not exist.")
    


In [10]:
check_for_file(file_path_edited)

The file does not exist.


In [11]:
for t in texts:
    with open(file_path_edited, 'w', encoding='utf-8') as file:
        file.write(t)
    with open(file_path_edited, 'r', encoding='utf-8') as file:
        text_read = file.read()
    print(text_read == t)

True
True
True
True
True


Watch out for this character as it is very awkward;

In [ ]:
print("first print")
print(ord("\x8e"))
print("second print")
print(chr(142))

Identify invalid characters using string:  Don't use this one as the later one is better.

In [ ]:

# Define a string containing only valid characters (alphanumeric + punctuation)
valid_chars = string.ascii_letters + string.digits + string.punctuation + " " = "\n"

# Example text with some invalid characters
#test_text = "This is an example text with invalid characters like Ò, ë, etc."
test_text = text
# Step 1: Identify invalid characters
invalid_chars = set([char for char in test_text if char not in valid_chars])

# Display the invalid characters found
print(f"Invalid characters found: {invalid_chars}")

# Step 2: Create a dictionary or list of tuples for replacement
# For simplicity, let's replace invalid characters with '?'
replacement_dict = {char: '?' for char in invalid_chars}

# If you want to create a list of tuples instead:
replacement_list = [(char, '?') for char in invalid_chars]

# Step 3: Replace invalid characters in the text
corrected_text = test_text
for char, replacement in replacement_dict.items():
    corrected_text = corrected_text.replace(char, replacement)

# Display the corrected text
#print(f"Corrected text: {corrected_text}")

# Output
print("Replacement dictionary:", replacement_dict)
print("Replacement list of tuples:", replacement_list)


Identify invalid characters using regular expressions (re):


In [12]:
# Define the pattern for valid characters (alphanumeric, spaces, and basic punctuation)
valid_char_pattern = re.compile(r'[a-zA-Z0-9\s.:;,?!\'"()£\-—…&]')

re_chars = len(valid_encodings) * [0]

for enc, encoding_index in enumerate(valid_encodings):
    test_text = texts[enc]

# Find invalid characters
    invalid_chars = set()
    for i, char in enumerate(test_text):
        if not valid_char_pattern.match(char):
            invalid_chars.add(char)

# Output the list of invalid characters and their positions
    invalid_char_positions = [(char, [i for i, c in enumerate(test_text) if c == char]) for char in invalid_chars]
    print(f"invalid characters using re for {encodings[encoding_index]}: {invalid_chars}")
#print(invalid_char_positions)
    re_chars[enc] = invalid_chars

invalid characters using re for utf-8: {'é', 'Ã', '§', '¨', '¼', '¢'}
invalid characters using re for latin-1: {'Â', '©', 'Ã', '§', '¨', '\x83', '¼', '¢'}
invalid characters using re for windows-1252: {'Â', '©', 'ƒ', 'Ã', '§', '¨', '¼', '¢'}
invalid characters using re for iso-8859-1: {'Â', '©', 'Ã', '§', '¨', '\x83', '¼', '¢'}
invalid characters using re for iso-8859-15: {'Â', '©', 'Ã', '§', 'š', '\x83', 'Œ', '¢'}


In [13]:
# Define valid characters using the string module
valid_chars = string.ascii_letters + string.digits + string.punctuation + "…—àêé \n\t"
valid_chars = ''.join([char for char in valid_chars if char not in "{}~$%"])
print (f"valid characters used by string: {string.ascii_letters + string.digits + ' ' + string.punctuation}")

string_chars = len(valid_encodings) * [0]

for enc, encoding_index in enumerate(valid_encodings):
    test_text = texts[enc]

    invalid_chars = set()
    for i, char in enumerate(test_text):
        if char not in valid_chars:
            invalid_chars.add(char)
    #invalid_chars.difference_update({"\n","\t"})

    invalid_char_positions = [(char, [i for i, c in enumerate(test_text) if c == char]) for char in invalid_chars]
    print(f"invalid characters found using string for {encodings[encoding_index]}: {invalid_chars}")
#print(invalid_char_positions)
    string_chars[enc] = invalid_chars



valid characters used by string: abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
invalid characters found using string for utf-8: {'Ã', '§', '¨', '\x0c', '¼', '¢'}
invalid characters found using string for latin-1: {'Â', '©', 'Ã', '§', '¨', '\x0c', '\x83', '¼', '¢'}
invalid characters found using string for windows-1252: {'Â', '©', 'ƒ', 'Ã', '§', '¨', '\x0c', '¼', '¢'}
invalid characters found using string for iso-8859-1: {'Â', '©', 'Ã', '§', '¨', '\x0c', '\x83', '¼', '¢'}
invalid characters found using string for iso-8859-15: {'Â', '©', 'Ã', '§', 'š', '\x0c', '\x83', 'Œ', '¢'}


In [28]:
for char in invalid_chars:
    print(ord(char)," ", char)


In [29]:
for char in re_chars:
    print(ord(char)," ", char)

224   à
234   ê
233   é


In [ ]:
#just a little test:
''.join([char for char in "abcdefgh" if char not in "cd"])

In [ ]:
print(valid_chars)
print(ord(valid_chars[68]))  #this is the ' after the & in the list
print(ord(valid_chars[-13])) #this is the ` before the { in the list
print(ord("'"))
print(ord('"'))
print(ord("`"))

In [15]:
def display_chars(chars, method, unprintable = 142):
    number = len(chars)
    invalid_chars = '  '.join(char for char in chars)
    unprintable_message = " + unprintable character" if chr(unprintable) in invalid_chars else ""
    print(f"{number} invalid chars found using {method}:  {invalid_chars}{unprintable_message}")

In [16]:
for enc, encoding_index in enumerate(valid_encodings):
    print(f"with encoding {encodings[encoding_index]}")
    
    display_chars(re_chars[enc], "re   ")
    display_chars(string_chars[enc], "string")
        
    re_only = re_chars[enc] - string_chars[enc]
    string_only = string_chars[enc] - re_chars[enc]
    
    print(f"\n{len(re_only)} chars in re but not in string:    {'  '.join(char for char in re_only)}")
    print(f"{len(string_only)} chars in string but not in re:    {'  '.join(char for char in string_only)}\n")

with encoding utf-8
6 invalid chars found using re   :  é  Ã  §  ¨  ¼  ¢
6 invalid chars found using string:  Ã  §  ¨    ¼  ¢

1 chars in re but not in string:    é
1 chars in string but not in re:    

with encoding latin-1
8 invalid chars found using re   :  Â  ©  Ã  §  ¨    ¼  ¢
9 invalid chars found using string:  Â  ©  Ã  §  ¨      ¼  ¢

0 chars in re but not in string:    
1 chars in string but not in re:    

with encoding windows-1252
8 invalid chars found using re   :  Â  ©  ƒ  Ã  §  ¨  ¼  ¢
9 invalid chars found using string:  Â  ©  ƒ  Ã  §  ¨    ¼  ¢

0 chars in re but not in string:    
1 chars in string but not in re:    

with encoding iso-8859-1
8 invalid chars found using re   :  Â  ©  Ã  §  ¨    ¼  ¢
9 invalid chars found using string:  Â  ©  Ã  §  ¨      ¼  ¢

0 chars in re but not in string:    
1 chars in string but not in re:    

with encoding iso-8859-15
8 invalid chars found using re   :  Â  ©  Ã  §  š    Œ  ¢
9 invalid chars found using string:  Â 

In [ ]:
print(ord("é"))
a = "eé".encode('utf-8')
print(len(a))

In [23]:
def compare_texts(text1, text2):
    if len(text1) != len(text2):
        #raise ValueError("Strings must be of the same length!")
        
# Create a list of tuples with differing characters
        differences = [(char1, char2) for char1, char2 in zip(text1, text2) if char1 != char2]

# Print the list of tuples
        print(differences[:100])



In [24]:
def find_first_difference(text_1, text_2):
    i = 0
    max_length = min(len(text_1),len(text_2))
    while ((text_1[:i] == text_2[:i]) and (i < max_length)):
        i = i + 1
    if i < max_length:
        print (f"first text, position {i} is {text_1[i]}")
        print(text_1[i:i+20])
        print (f"second text, position {i} is {text_2[i]}")
        print(text_2[i:i+20])
    else:
        print("no differences up to end of the smaller text")

In [18]:
# compare_texts(texts[0], texts[1])
# find_first_difference(texts[0],texts[1])

Now we are going to display all the invalid characters in their context to decide what to do about them. We will just do it for one of the versions as they are all the same.

In [32]:
#  This is an updated version that checks against re AND string characters

def display_invalid_chars(test_text, window_size = 200, max_to_display = 8):
# Find all invalid characters and their positions
    #valid_chars = string.ascii_letters + string.digits + string.punctuation + " —…()£-&\n\t"
    #print(valid_chars)
    invalid_char_positions = [(i, char) for i, char in enumerate(test_text) if char not in valid_chars or char in re_chars]
                          
    icp_list_sorted = sorted(invalid_char_positions,key=lambda x: x[1])
    
    for pos, char in icp_list_sorted:
        start = max(pos - window_size // 2, 0)
        end = min(pos + window_size // 2, len(test_text))
        window = test_text[start:end]
    
    # Highlight the invalid character in the window
        highlighted_window = window[:pos-start] + f"[{char}]" + window[pos-start+1:]
        highlighted_window = highlighted_window.replace("\n", " ")
        highlighted_window = highlighted_window.replace("\t", " ")
        print(f"Invalid character '{char}' ({ord(char)}) at position {pos}:\n{highlighted_window}\n")




In [123]:
def display_text_in_context(source_text,target_text,window = 50, repeat = False):
    ind = source_text.find(target_text)
    if not repeat:
        if ind != -1:
            window = source_text[max(0, ind - window): ind + window + len(target_text)]
            window = window.replace("\n", " ")
            window = window.replace("\t", " ")
            print(window)
        else:
            print("not found")
    else:
        if ind == -1:
            print("not found")
        while True:
            ind = source_text.find(target_text, ind)
            if ind == -1:
                break
            start_window = max(0, ind - window)
            end_window = min(len(source_text), ind + len(target_text) + window)
            window_text = source_text[start_window:end_window]
            print(ind, window_text)
            ind += 1  # Move to the next position
        
        
        
        
#        while ind != -1:
#            display_window = source_text[max(0, ind - window): ind + window + len(target_text)]
#            display_window = display_window.replace("\n", " ")
#            display_window = display_window.replace("\t", " ")
#            print(display_window)
#            source_text = source_text[ind:]


In [24]:
# display_invalid_chars(texts[1][0:5000])

In [32]:
display_invalid_chars(texts[2])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [33]:
display_invalid_chars(texts[3])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
print("€" in texts[3])
print(texts[3][246245])

In [61]:
# Function to find and replace specified strings in a text
def replace_strings_with_context(text, replacements, window=10, silent = False):
    """
    Replaces specified strings within a text and prints the context around each replacement.
    
    Parameters:
    - text: The original string where replacements are done.
    - replacements: A dictionary where keys are the substrings to find, and values are their replacements.
    - window: Number of characters to show before and after the replaced string for context.
    
    Returns:
    - A modified string with the replacements applied.
    """
    counter = 0
    for old, new in replacements.items():
        index = text.find(old)
        while index != -1:
            # Extract context around the found string
            start = max(index - window, 0)
            end = min(index + len(old) + window, len(text))
            before = text[start:index]
            after = text[index + len(old):end]
            before = before.replace("\n", " ")
            before = before.replace("\t", " ")
            after = after.replace("\n", " ")
            after = after.replace("\t", " ")

            # Print before, replaced, and after strings
            if not silent:
                print(f"Before: {before}[{old}]{after}")
                print(f"After:  {before}[{new}]{after}\n")

            # Replace the string in the text
            text = text[:index] + new + text[index + len(old):]
            counter += 1
            
            # Find the next occurrence of the old string
            index = text.find(old, index + len(new))

    print(f"made {counter} changes")
    return text




In [ ]:
print(ord("b"))
print(chr(98))
print(ord("Ž"))

In [24]:
replacements = dict()
# replacements["t}:le"] = "the"
# replacements["up o~ the"] = "up on the"
# replacements["the ~ort"] = "the sort"
# replacements["than ~ might"] = "than might"
# replacements["the house~"] = "the house."
# replacements["while ~earlier"] = "while earlier"
# replacements["meetings ~once"] = "meetings once"
# replacements["Ž"] = "é"
# replacements["mean- it"] = "mean it"
# replacements["¤"] = "s"
# replacements["Ð"] = "-"
# replacements["É"] = '-'
# replacements["Ó"] = '"'
# replacements["Ò"] = '"'
# replacements["Ô"] = '"'
# replacements["Rover.Ó I thanked"] = "Rover  I thanked"
# replacements['Rover." I thanked'] = "Rover  I thanked"
# replacements["Õ"] = "'"
# replacements[chr(136)] = "à"
# replacements[chr(142)] = "é"
# replacements[chr(144)] = "ê"
# replacements["Ñ"] = "-"
replacements[chr(12)] = ""
# replacements["%egan"] = "began"
# replacements["t4arijço"]="Sachiko"
replacements[chr(8217)] = "'"
replacements[chr(8216)] = '"'
replacements[chr(8220)] = '"'
replacements[chr(8221)] = '"'
# replacements["*"] = '"'
# replacements["AH the same"] = "Ah the same"
# replacements["•"] = ""
replacements["AdelÃ¨"] = "Adela"
replacements["faÃ§ade"] = "facade"
replacements["pÃ¢tisserie"] = "patisserie"
replacements["JÃ¼rgen"] = "Jorgen"


for key, value in replacements.items():
        print(f"{key}: {value}")

: 
’: '
‘: "
“: "
”: "
AdelÃ¨: Adela
faÃ§ade: facade
pÃ¢tisserie: patisserie
JÃ¼rgen: Jorgen


In [27]:
text_edited = replace_strings_with_context(texts[0],replacements, silent = True)

made 356 changes


Now see if there are any problems left:

In [28]:
display_invalid_chars(text_edited)


In [31]:
display_text_in_context(text_edited, "The End", window = 40, repeat = True)
display_text_in_context(text_edited, "PART", window = 180, repeat = True)

1159196 egan making my way back to my seat.




The End

2 

PART I


The taxi driver seemed embarrassed to find there was no one - not even a clerk behind the reception desk
- waiting to welcome me. He wandered across the deserted lobby, perhap
327742  was no real need to hear any more - given adequate time, the Kazan was very
obviously within his capabilities - and I began to walk slowly away in the direction of the lobby.




PART II




11




I was woken by the ringing of the telephone on the bedside cabinet. My first thought was that I had again
been disturbed after only a few minutes, but then I saw fro
630031 udden wish that I was there at this very moment. I began to
walk again, over the bridge and into the dark street, resolving to call on her in the morning at the first
opportunity.
PART III




21




I awoke to find bright sunlight pouring in through the vertical blinds and was seized by the panicky
feeling I had let far too much of the morning slip by. But then
892758 ne.

In [104]:
print(ord("\n"))

10


In [32]:
titles = list()
# titles.append(("PART ONE", "Chapter One"))
# titles.append(("PART TWO", "Chapter Four"))
# titles.append(("PART THREE", "Chapter Ten"))
# titles.append(("PART FOUR", "Chapter Twelve"))
# titles.append(("PART FIVE", "Chapter Fourteen"))
# titles.append(("PART SIX", "Chapter Sixteen"))
# titles.append(("PART SEVEN", "Chapter Twenty-three"))
# titles.append(("Part I", "Chapter One"))
# titles.append(("Part II", "Chapter Six"))
# titles.append(("Part III", "Gawain's First Reverie"))                            
# titles.append(("Part IV", "Chapter Fifteen"))



for edit_start, edit_end in titles:
    print(edit_start, edit_end)


In [45]:
def remove_chunks(text, chunks, remove_all = False, buffer = 100):
    # removes chunks defines by a list of tuples, start of chunk to end of chunk, removes the second part only if remove_all is set to True
    print("text length:", len(text))
    for edit_start, edit_end in chunks: 
        ind1 = text.find(edit_start)
        ind2 = text.find(edit_end)
        if remove_all:
            ind2 += len(edit_end)
        if ind2 > ind1 and ind2 < ind1 + 100 and ind1 != -1 and ind2 != -1:
            print (f"removing {edit_start} to {edit_end}")
            text = text[:ind1] + text[ind2:]
        else:
            print(f"{edit_start} to {edit_end} not removed")
    print("text length reduced to: ", len(text))
    return text


In [90]:
#TEST
tt = "The quick brown fox jumps over the lazy dog"
tt1 = [("quick", "the ")]
tt2 = remove_chunks(tt, tt1, remove_all = True)
print(tt)
print(tt2)

removing quick to the 
The quick brown fox jumps over the lazy dog
The lazy dog


In [38]:
#text_edited_1 = remove_chunks(text_edited, titles, remove_all = True)

In [65]:
tidy_up = dict()
tidy_up["  "] = " " 
tidy_up["Chapter Sixteen"] = ""
tidy_up["Chapter Seventeen"] = ""
tidy_up["Chapter Eighteen"] = ""
tidy_up["Chapter Nineteen"] = ""
tidy_up["Chapter Two"] = ""
tidy_up["Chapter Three"] = ""
tidy_up["Chapter Four"] = ""
tidy_up["Chapter Five"] = ""
tidy_up["Chapter Six"] = ""
tidy_up["Chapter Seven"] = ""
tidy_up["Chapter Eight"] = ""
tidy_up["Chapter Nine"] = ""
tidy_up["Chapter Ten"] = ""
tidy_up["Chapter Eleven"] = ""
tidy_up["Chapter Twelve"] = ""
tidy_up["Chapter Thirteen"] = ""
tidy_up["Chapter Fourteen"] = ""
tidy_up["Chapter Fifteen"] = ""
tidy_up["Chapter Twenty-one"] = ""
tidy_up["Chapter Twenty-two"] = ""
tidy_up["Chapter Twenty"] = ""
tidy_up["PART I"] = ""
tidy_up["PART II"] = ""
tidy_up["PART III"] = ""
tidy_up["PART IV"] = ""
tidy_up["The End"] = ""


In [66]:
text_edited_2 = replace_strings_with_context(text_edited_2,tidy_up, silent = True)

made 5 changes


In [61]:
text_edited_2 = replace_strings_with_context(text_edited_2,tidy_up, silent = True)

made 0 changes


In [37]:
display_text_in_context(text_edited_2, "part")

e able to do the job. It's a delusion that's been particularly nurtured in this town. Call it a local my
not found


In [62]:
counter = 0
for char in text_edited_2:
    if char != " ":
        counter +=1
print (counter)

936176


In [67]:
with open(file_path_edited_2, 'w', encoding='utf-8') as f:
    f.write(text_edited_2)

Check that saving and loading the corrected file doesn't change it.  There will be a change when it encounters é because it takes 2 bytes.  Up to that point, there should be no differences.

In [68]:
with open(file_path_edited_2, 'r', encoding='utf-8') as f:
    text = f.read()

In [69]:
text == text_edited_2

True

In [114]:
text_edited.find("é")

-1

In [113]:
print(len(text), len(text_edited_2))

578625 578369


In [115]:
text_edited_2.find(chr(8216))

-1

In [120]:
text_edited_1.find(chr(8230))

19068

In [122]:
print(text_edited_1[200:230])
print(text_edited_1[19000:19120])
      

nese name, and I— perhaps out 
f the
city.'
"Oh really? I didn't realize that. You lived there with…with friends?"
Sachiko stopped pouring and looked a


In [70]:

# Sample text with hyphenated words across lines
t = texts[0]
# Use regex to find words that are hyphenated across lines
# Match sequences where a hyphen is at the end of a line, followed by a newline, and then continued with a word
hyphenated_words = re.findall(r"(\w+)-\n(\w+)", t)

# Print the hyphenated words
for first_part, second_part in hyphenated_words:
    print(f"Found broken word: {first_part}-{second_part}")
print(len(hyphenated_words))
print(len(t))


Found broken word: not-so
Found broken word: so-called
Found broken word: mock-pompous
Found broken word: near-darkness
Found broken word: pudgy-faced
Found broken word: sun-filled
Found broken word: full-chested
Found broken word: nervous-looking
Found broken word: red-nosed
Found broken word: have-no
Found broken word: half-way
Found broken word: high-pitched
Found broken word: mid-sentence
Found broken word: self-possessed
Found broken word: half-finished
Found broken word: early-morning
16
1159560
